In [ ]:
import rasterio
from datetime import datetime
import geopandas as gpd
from glob import glob
import pandas as pd
import os

In [ ]:
fordri_dir = "/work/tadesse/beichen/ForDRI/Data/Op_ForDRI/"
fordri_flist = glob(fordri_dir+"*.tif")
fordri_flist.sort()

## Extract ForDRI values at tree-ring sites in the US

In [66]:
# western US
west_tr_shp_dir = "/work/tadesse/beichen/ForDRI/Data/Shapefiles/Tree_site_info_overlapped_7yrs_westernUS.shp"
west_tr_shp = gpd.read_file(west_tr_shp_dir)
west_tr_shp = west_tr_shp[['Collection','Northernmo', 'Westernmos']] # latitude and longitude
west_tr_shp.index = range(len(west_tr_shp))
west_tr_coords = [(x,y) for x, y in zip(west_tr_shp.Westernmos,west_tr_shp.Northernmo)]  # longitude and latitude

In [67]:
# eastern US
east_tr_shp_dir = "/work/tadesse/beichen/ForDRI/Data/Shapefiles/Tree_site_info_overlapped_7yrs_easternUS.shp"
east_tr_shp = gpd.read_file(east_tr_shp_dir)
east_tr_shp = east_tr_shp.drop_duplicates() # drop diplicates
# east_tr_shp = east_tr_shp.loc[east_tr_shp['Collection']!='AR078',:]
east_tr_shp = east_tr_shp[['Collection','Northernmo', 'Westernmos']]
east_tr_shp.index = range(len(east_tr_shp))
east_tr_coords = [(x,y) for x, y in zip(east_tr_shp.Westernmos,east_tr_shp.Northernmo)]

In [ ]:
# extracting ForDRI at the tree ring sites
for fordri in fordri_flist:
    ts = os.path.basename(fordri).split("_")[1][:-4]
    ts_week = datetime.strptime(ts,"%Y%m%d").strftime("%Y%W")
    with rasterio.open(fordri) as src:
        west_tr_shp[ts_week] = [x[0] for x in src.sample(west_tr_coords)]
        east_tr_shp[ts_week] = [x[0] for x in src.sample(east_tr_coords)]

In [70]:
west_tr_shp.to_csv("/work/tadesse/beichen/ForDRI/Data/Tree_Ring_Data/NEW_tree_ring_sites_fordri_westernUS.csv", index=None)
east_tr_shp.to_csv("/work/tadesse/beichen/ForDRI/Data/Tree_Ring_Data/NEW_tree_ring_sites_fordri_easternUS.csv", index=None)

## Extract ForDRI values at Bowen Ratio (AmeriFlux) sites in the US

In [61]:
br_dir ="/work/tadesse/beichen/ForDRI/Data/Shapefiles/Filtered_AmeriFlux_Sites.shp"
br_shp = gpd.read_file(br_dir)
br_shp = br_shp[['Site ID','Latitude (','Longitude']]
br_shp.columns=["SiteID","Lat","Long"]
br_shp.index = range(len(br_shp))
br_coords = [(x,y) for x, y in zip(br_shp.Long,br_shp.Lat)]

In [ ]:
# extracting ForDRI at the AmeriFlux sites
for fordri in fordri_flist:
    ts = os.path.basename(fordri).split("_")[1][:-4]
    ts_week = datetime.strptime(ts,"%Y%m%d").strftime("%Y%W")
    with rasterio.open(fordri) as src:
        br_shp[ts_week] = [x[0] for x in src.sample(br_coords)]

In [65]:
br_shp.to_csv("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/ameriflux_sites_fordri_US.csv", index=None)

In [ ]:
stdbr_dir = "/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/Bowen_Ratio/"
stdbr_list = glob(stdbr_dir+"*.csv")
stdbr_list.sort()
for stdbr in stdbr_list:
    fname = os.path.basename(stdbr)
    df = pd.read_csv(stdbr,index_col=0)
    start = df.index[0]
    end = df.index[-1]
    print(start,end)
    if start < 200301:
        start = 200301
    if end > 202352:
        end = 202352
    df = df.loc[200301:202352,:]
    df.to_csv("/work/tadesse/beichen/ForDRI/Data/New_Bowen_Ratio/ForDRI_stdBR/"+fname)
    